# Задание 1 (5 балла)

Имплементируйте алгоритм Леска (описание есть в семинаре) и оцените качество его работы на датасете `data/corpus_wsd_50k.txt`

В качестве метрики близости вы должны попробовать два подхода:

1) Jaccard score на множествах слов (определений и контекста)
2) Cosine distance на эмбедингах sentence_transformers

В качестве метрики используйте accuracy (% правильных ответов). Предсказывайте только многозначные слова в датасете

Контекст вы можете определить самостоятельно (окно вокруг целевого слова или все предложение). Также можете поэкспериментировать с предобработкой для обоих методов.

В качестве контекста было выбрано предложение, в котором находится слово.

Окончательный вариант решения, который не выдает ошибок, но в google collab время выполнения этого кода около 4х часов:

In [ ]:
import codecs
import nltk
from nltk.corpus.reader import Synset
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from sklearn.metrics.pairwise import cosine_similarity

!mkdir data
!wget https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/corpus_wsd_50k.txt.zip -P data
!unzip -o data/corpus_wsd_50k.txt.zip -d data/

!python -m pip install torch torchvision torchaudio
!python -m pip install sentence_transformers transformers accelerate -U
from sentence_transformers import SentenceTransformer

nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')


def jaccard_score(definition: set, context: set):
    intersection = definition & context
    union = definition | context
    jaccard = len(intersection) / len(union)
    return jaccard


def cosine_distance_score(definition, context):
    definition_embedding = embed(' '.join(definition))
    context_embedding = embed(' '.join(context))
    cosine_distance = cosine_similarity(context_embedding.reshape(1, -1), definition_embedding.reshape(1, -1))
    return cosine_distance


def lesk_algorithm(context: str, word: str) -> tuple[Synset | None, Synset | None]:
    synsets = wn.synsets(word)
    context_set = set(context.split())

    jaccard_best_synset = None
    cosine_dist_best_synset = None
    jaccard_best_score = 0
    cosine_dist_best_score = 0

    for synset in synsets:
        definition = set(synset.definition().split())
        jaccard = jaccard_score(definition, context_set)
        cosine_dist = cosine_distance_score(definition, context_set)
        if jaccard > jaccard_best_score:
            jaccard_best_synset = synset
            jaccard_best_score = jaccard
        if cosine_dist > cosine_dist_best_score:
            cosine_dist_best_synset = synset
            cosine_dist_best_score = cosine_dist

    return jaccard_best_synset, cosine_dist_best_synset


def accuracy_metric(jaccard_cosine_tuple: tuple[Synset | None, Synset | None], original_synset: Synset):
    jaccard_is_correct = 0
    cosine_is_correct = 0
    if jaccard_cosine_tuple[0] is not None and jaccard_cosine_tuple[0].name() == original_synset.name():
        jaccard_is_correct += 1
    if jaccard_cosine_tuple[1] is not None and jaccard_cosine_tuple[1].name() == original_synset.name():
        cosine_is_correct += 1

    return jaccard_is_correct, cosine_is_correct


corpus_wsd = []
file = codecs.open('data/corpus_wsd_50k.txt', 'r', 'utf_8_sig')
corpus = file.read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])
file.close()

english_stopwords = set(stopwords.words('english'))

# модель эмбеддинга
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embed = model.encode

sentences_data = []
for corpus_sentence in corpus_wsd:
    sentence = ''
    polysemous_sense = ''
    polysemous_word = ''
    for corpus_word in corpus_sentence:
        if len(corpus_word) < 3:
            break

        sense, _, word = corpus_word
        if '%' in sense and not polysemous_sense:
            polysemous_sense = sense
            polysemous_word = word
        else:
            sentence += ''.join(word) + ' '

    sentences_data.append((sentence, polysemous_sense, polysemous_word))

total = len(corpus_wsd)
jaccard_correct = 0
cosine_correct = 0

for sentence_data in sentences_data:
    sentence, sense, word = sentence_data
    if not sense:
        continue
    jaccard_cosine_tuple = lesk_algorithm(sentence, word)

    original_synset = wn.lemma_from_key(sense).synset()
    jaccard, cosine = accuracy_metric(jaccard_cosine_tuple, original_synset)
    jaccard_correct += jaccard
    cosine_correct += cosine

jaccard_metric = jaccard_correct / total * 100
cosine_metric = cosine_correct / total * 100

print('jaccard ' + str(jaccard_metric) + ' cosine ' + str(cosine_metric))

Изначальный код до дорабатывания его в google collab, который не считает accuracy, но выдает jaccard_score и cosine_distance_score для одного слова, если вводить его вручную:

In [3]:
import codecs
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [4]:
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\eliza\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eliza\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\eliza\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
corpus_wsd = []
file = codecs.open('data/corpus_wsd_50k.txt', 'r', 'utf_8_sig')
corpus = file.read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])
file.close()

In [6]:
english_stopwords = set(stopwords.words('english'))

# модель эмбеддинга
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embed = model.encode

C:\Users\eliza\PycharmProjects\compling_nlp_hse_course\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [25]:
def sentence_and_word(corpus_wsd):
    sentences_and_words = []

    for sentence_data in corpus_wsd:
        # Собираем предложение из всех слов в sentence_data
        # sentence = ' '.join([word_data[2] for word_data in sentence_data])

        # Для каждого слова в предложении проверяем, многозначное оно или нет
        for word_data in sentence_data:
            sense, lemma, word = word_data
            if '%' in sense:  # Если это многозначное слово
                # Добавляем к результатам предложение и это многозначное слово
                sentences_and_words.append(sentence_data, word, sense)

    return sentences_and_words

In [9]:
# Эта функция разбивает на токены
def preprocess(text: list):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha() and token not in english_stopwords]
    return ' '.join(tokens)

In [10]:
def jaccard_score(definition: set, context: set):
    intersection = definition & context
    union = definition | context
    jaccard = len(intersection) / len(union)
    return jaccard

In [75]:
def cosine_distance_score(definition, context):
    # Надо превратить определение и контекст в эмбеддинги и посчитать их
    definition_embedding = embed(' '.join(definition))
    context_embedding = embed(' '.join(context))
    cosine_distance = cosine_similarity(context_embedding.reshape(1, -1), definition_embedding.reshape(1, -1))
    return cosine_distance

In [77]:
def lesk_algorithm(context: str, word: str):
    synsets = wn.synsets(word)
    if len(synsets) <= 1:
        return 'Not polysemous word'
    
    context_set = set(context.split())
    
    result = []
    for synset in synsets:
        definition = set(synset.definition().split())
        jaccard = jaccard_score(definition, context_set)
        cosine = cosine_distance_score(definition, context_set)
        print('jaccard: ' + str(jaccard) + ' cosine: ' + str(cosine))

In [21]:
# def accuracy_metric(corpus_wsd):
#     sentences_and_words = sentence_and_word(corpus_wsd)
# 
#     correct = 0
#     total = 0
# 
#     for sentence_data, word, true_sense in sentences_and_words:
#         # Применяем алгоритм Леска
#         predicted_sense = lesk_algorithm(sentence_data, word)
# 
#         if predicted_sense and predicted_sense.name() == true_sense:
#             correct += 1
#         total += 1
# 
#     return correct / total * 100 if total > 0 else 0

In [78]:
# accuracy = accuracy_metric(corpus_wsd)
# print(f'Accuracy: {accuracy:.2f}%')

sentence_list = []
for item in corpus_wsd:
    sentence = ''
    word = ''
    for word_list in item:
        word = ''.join(word_list[2])
        sentence += word + ' '
    sentence_list.append(sentence)
    break
        
word = 'long'
predicted_sense = lesk_algorithm(sentence_list[0], word)


jaccard: 0.0 cosine: [[0.09104943]]
jaccard: 0.029411764705882353 cosine: [[0.12430339]]
jaccard: 0.034482758620689655 cosine: [[0.0590357]]
jaccard: 0.043478260869565216 cosine: [[0.01143938]]
jaccard: 0.0 cosine: [[0.13970378]]
jaccard: 0.038461538461538464 cosine: [[-0.0247157]]
jaccard: 0.08333333333333333 cosine: [[0.08094941]]
jaccard: 0.0 cosine: [[0.13382748]]
jaccard: 0.047619047619047616 cosine: [[0.18524656]]
jaccard: 0.0 cosine: [[0.03750885]]
jaccard: 0.0 cosine: [[0.11933612]]
jaccard: 0.0 cosine: [[0.10371695]]


# Задание 2 (5 балла)
Попробуйте разные алгоритмы кластеризации на датасете - `https://github.com/nlpub/russe-wsi-kit/blob/initial/data/main/wiki-wiki/train.csv`

Используйте код из семинара как основу. Используйте ARI как метрику качества.

Попробуйте все 4 алгоритма кластеризации, про которые говорилось на семинаре. Для каждого из алгоритмов попробуйте настраивать гиперпараметры (посмотрите их в документации). Прогоните как минимум 5 экспериментов (не обязательно успешных) с разными параметрами на каждый алгоритме кластеризации и оцените: качество кластеризации, скорость работы, интуитивность параметров.

Помимо этого также выберите 1 дополнительный алгоритм кластеризации отсюда - https://scikit-learn.org/stable/modules/clustering.html , опишите своими словами принцип его работы  и проделайте аналогичные эксперименты. 

In [2]:
import time

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/nlpub/russe-wsi-kit/initial/data/main/wiki-wiki/train.csv', sep='\t')

In [5]:
df.head(100)

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,1,замок,1,NaN,"0-5, 339-344",замок владимира мономаха в любече . многочисле...
1,2,замок,1,NaN,"11-16, 17-22, 188-193","шильонский замок замок шильйон ( ) , известный..."
2,3,замок,1,NaN,299-304,проведения архитектурно - археологических рабо...
3,4,замок,1,NaN,111-116,"топи с . , л . белокуров легенда о завещании м..."
4,5,замок,1,NaN,"134-139, 262-267",великий князь литовский гедимин после успешной...
...,...,...,...,...,...,...
95,96,замок,1,NaN,"163-168, 213-218, 364-369",без каминов и очагов . у входа в башню было по...
96,97,замок,1,NaN,"221-226, 276-281",v . в том сражении шотландцы потерпели сокруши...
97,98,замок,1,NaN,"0-5, 16-21, 179-184",замок данноттар замок данноттар ( ) расположен...
98,99,замок,1,NaN,232-237,. благодаря поэме байрона заключение бонивара ...


In [6]:
grouped_df = df.groupby('word')[['word', 'context', 'gold_sense_id']]

In [47]:
from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation, AgglomerativeClustering, OPTICS
import numpy as np
from sklearn.metrics import adjusted_rand_score

from IPython.display import Image
from IPython.core.display import HTML

In [8]:
from sentence_transformers import SentenceTransformer

C:\Users\eliza\PycharmProjects\compling_nlp_hse_course\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [9]:
# embedding model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embed = model.encode

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\eliza\PycharmProjects\compling_nlp_hse_course\.venv\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\eliza\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

C:\Users\eliza\PycharmProjects\compling_nlp_hse_course\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Kmeans

In [68]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = KMeans(7)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.11446408920413326
Время выполнения: 165.31 секунд


In [69]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    # cluster = AffinityPropagation(damping=0.9)
    cluster = KMeans(2)
#     cluster = DBSCAN(min_samples=1, eps=0.1)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

0.06806698150265096
Время выполнения: 149.71 секунд


In [70]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    # cluster = AffinityPropagation(damping=0.9)
    cluster = KMeans(4)
#     cluster = DBSCAN(min_samples=1, eps=0.1)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

0.07350251244932049
Время выполнения: 151.09 секунд


In [71]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    # cluster = AffinityPropagation(damping=0.9)
    cluster = KMeans(10)
#     cluster = DBSCAN(min_samples=1, eps=0.1)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

0.06485713902036974
Время выполнения: 148.89 секунд


In [72]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    # cluster = AffinityPropagation(damping=0.9)
    cluster = KMeans(5)
#     cluster = DBSCAN(min_samples=1, eps=0.1)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

0.038730501929247436
Время выполнения: 151.44 секунд


Самый лучший результат для этой метрики при параметре 7 (0.11446408920413326), при этом алгоритм работает самое долгое время, по сравнению с результатами при других параметрах. Однако я несколько раз запускала алгоритм с одинаковым параметром и каждый раз результаты были разные. Вероятно, это происходит потому, что при каждом запуске центроиды выбираются рандомно. Это так же прибавляет сложности поиску оптимального параметра. Не совсем интуитивно понятно, какое количество кластеров выбрать оптимальным, так как результаты и при небольшом количестве кластеров (2), и при большом количестве кластеров (10) примерно одинаковые (0.06806698150265096 vs 0.06485713902036974). При этом самый худший результат получился при параметре 5 (0.038730501929247436), хотя при параметрах 4 (0.07350251244932049) и 3 (0.07963918459393977 - взят из лекции), результаты уже лучше. Это опять же объясняется рандомным выбором центроидов.

Если учитывать результаты параметров, которые отсутствуют в ответе, но были выполнены, то с увеличением количества кластеров (начиная от 9-10), результаты кластеризации становятся хуже, так же как и если брать результаты кластеризации при параметре 1 и 2. В одной из итераций результат для параметра 2 оказался примерно 0.00298, а для параметра 1 вообще 0.0.

### DBSCAN

In [73]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = DBSCAN(min_samples=1, eps=0.1)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.001053019960000099
Время выполнения: 150.77 секунд


In [75]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = DBSCAN(min_samples=5, eps=0.3)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: -0.02349153051130162
Время выполнения: 151.19 секунд


In [10]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = DBSCAN(min_samples=4, eps=0.68)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.07402644018324375
Время выполнения: 160.32 секунд


In [11]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = DBSCAN(min_samples=8, eps=0.5)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.012476507850342128
Время выполнения: 153.11 секунд


In [12]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = DBSCAN(min_samples=1, eps=0.52)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.11842783110378406
Время выполнения: 170.10 секунд


In [13]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = DBSCAN(min_samples=2, eps=0.52)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.10894038562828764
Время выполнения: 154.23 секунд


In [14]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = DBSCAN(min_samples=1, eps=0.61)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.13372367481954406
Время выполнения: 153.53 секунд


In [15]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = DBSCAN(min_samples=2, eps=0.61)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.13259354738443666
Время выполнения: 151.75 секунд


Для этого датасета при использовании метрики DBSCAN наиболее точные результаты получаются при уменьшении количества объектов и радиусе между 0.52 и 0.61. Также в последних тестах, где сравниваются показатели при количестве объектов 1 и 2 и радиусе 0.51 и 0.61 (и их комбинации), можно заметить, что для более точного результата требуется большее количество времени, а также при уменьшении минимального количества объектов времени тоже затрачивается больше. Настройка параметров интуитивно понятна, особым плюсом стоит здесь отметить то, что метрика сама определяет количество кластеров на основе плотности. Самый сложный момент - это понять, какой радиус должен быть и сколько минимальных объектов должно быть.

DBSCAN выдает более точные результаты по сравнению с Kmeans для этого датасета, но при этом увеличивается скорость работы программы. Более точный результат кластеризации скорее всего получается потому, что данные имеют сложную форму, с которой лучше работает DBSCAN, а не сферическую, на что нацелен Kmeans.

### Affinity Propagation

In [16]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = AffinityPropagation(damping=0.9)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.05297560306165972
Время выполнения: 151.98 секунд


In [20]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = AffinityPropagation(damping=0.52)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.042740969848549505
Время выполнения: 183.41 секунд


In [21]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = AffinityPropagation(damping=0.7)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.04150923434928373
Время выполнения: 162.80 секунд


Для Affinity Propagation тесты с изменением параметра damping выдавали результат не выше 0.05297560306165972, поэтому захотелось попробовать настроить preference.


In [23]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = AffinityPropagation(preference=10, damping=0.52)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.0
Время выполнения: 153.23 секунд


In [25]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = AffinityPropagation(preference=-50, damping=0.9)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.0
Время выполнения: 172.30 секунд


В Jupiter Notebook при добавлении параметра preference метрика выдает 0.0, в Google Collab выдавалось при любых damping и preference значение 0.05297560306165972.

При использовании Affinity Propagation время работы программы большее, чем при использовании Kmeans и DBSCAN. Точность кластеризации так же ниже. Возможно, датасету нужна дополнительная предобработка, так как все предыдущие метрики тоже не выдавали очень точных результатов (близких к 1).

Немного более точные результаты выдавались при параметре damping стремящемся к 1, этот параметр достаточно понятен.
В теории параметр preference при низких значениях должен был уменьшать количество кластеров, а при больших значениях, наоборот, увеличивать. Но не совсем интуитивно понятно, какое количество кластеров оптимальное (от какого значения надо уменьшать или увеличивать количество кластеров).

### Agglomerative clustering

In [30]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = AgglomerativeClustering(n_clusters=2)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: -0.011976265536517934
Время выполнения: 175.53 секунд


In [31]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = AgglomerativeClustering(n_clusters=8)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.057641599566266404
Время выполнения: 176.19 секунд


In [36]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = AgglomerativeClustering(n_clusters=9)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.06308052666462462
Время выполнения: 158.21 секунд


In [33]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = AgglomerativeClustering(n_clusters=5)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.03379296274962468
Время выполнения: 174.61 секунд


In [34]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = AgglomerativeClustering(n_clusters=20)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.04406294885576405
Время выполнения: 163.42 секунд


При использовании только параметра количества кластеров для Agglomerative clustering результаты кластеризации не очень точные. Лучший результат выдается при количестве кластеров 9.

Попробуем косинусное расстояние в качестве метрики расстояния (вместо базового евклидового расстояния), для этого еще нужно заменить параметр linkage (который по умолчанию равен ward)

In [38]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = AgglomerativeClustering(n_clusters=9, metric="cosine", linkage='average')
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.1245453538775664
Время выполнения: 170.63 секунд


Результат уже получается намного лучше, попробуем другие параметры linkage.

In [39]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = AgglomerativeClustering(n_clusters=9, metric="cosine", linkage='complete')
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.06987009143297748
Время выполнения: 170.43 секунд


In [40]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = AgglomerativeClustering(n_clusters=9, metric="cosine", linkage='single')
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.15965864432739085
Время выполнения: 170.75 секунд


In [41]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = AgglomerativeClustering(n_clusters=9, metric="manhattan", linkage='average')
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.11781181319772398
Время выполнения: 167.58 секунд


In [42]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = AgglomerativeClustering(n_clusters=9, metric="manhattan", linkage='complete')
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.046212305114339
Время выполнения: 176.44 секунд


In [43]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = AgglomerativeClustering(n_clusters=9, metric="manhattan", linkage='single')
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.15965864432739085
Время выполнения: 169.01 секунд


Наилучший результат кластеризации для Agglomerative clustering получается при использовании параметров (n_clusters=9, metric="manhattan", linkage='single') и (n_clusters=9, metric="cosine", linkage='single'). Время выполнения программ выше, чем у предыдущих метрик, но при этом эта метрика выдает лучшие результаты кластеризации.
Параметры интуитивно понятны, главное - разобраться с количеством кластеров. 

### OPTICS

OPTICS - алгорит кластеризации, который поход на DBSCAN, однако, в отличие от DBSCAN, OPTICS позволяет выявлять кластеры с различной плотностью. Этот алгоритм упорядочивает точки данных по их плотностям, строя так называемый "reachability plot" (график достижимости), который можно использовать для идентификации кластеров с разной плотностью.

Основные параметры OPTICS:
min_samples: Минимальное количество точек, которые должны быть в радиусе eps от центральной точки, чтобы она считалась "ядром".
max_eps: Максимальное расстояние между двумя точками для их объединения в один кластер. При уменьшении максимального расстояния время работы программы будет уменьшаться.
xi: Порог, определяющий падение плотности, при котором OPTICS считает, что достигнут конец одного кластера и начинается другой.
metric: Расстояние, используемое для вычисления плотности (по умолчанию — minkowski).

In [48]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = OPTICS(min_samples=5, xi=0.05, min_cluster_size=0.1)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: -0.005376253758977646
Время выполнения: 162.57 секунд


In [50]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = OPTICS(min_samples=1.0, xi=0.05, min_cluster_size=0.1)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.0
Время выполнения: 157.78 секунд


In [51]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = OPTICS(min_samples=2, xi=0.05, min_cluster_size=0.1)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: -0.005376253758977646
Время выполнения: 159.83 секунд


In [54]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = OPTICS(min_samples=2, xi=0.52, min_cluster_size=None)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.016982890179909997
Время выполнения: 147.09 секунд


In [59]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = OPTICS(min_samples=2, xi=0.61, min_cluster_size=None)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.03095171203962383
Время выполнения: 149.22 секунд


In [63]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = OPTICS(min_samples=2, xi=0.79, min_cluster_size=None)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.07676424673416805
Время выполнения: 148.50 секунд


In [68]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = OPTICS(min_samples=2, xi=0.79, min_cluster_size=2)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.07676424673416805
Время выполнения: 145.96 секунд


In [69]:
start_time = time.time()

ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = OPTICS(min_samples=2, xi=0.79, min_cluster_size=3)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Время выполнения: {elapsed_time:.2f} секунд")

Метрика ARI: 0.01654322779148901
Время выполнения: 148.99 секунд


Время работы программы с использованием OPTICS меньше, чем при использовании Agglomerative clustering и Affinity Propagation.
Лучший результат получается при минимальном количестве экземпляров 2 и пороге падения плотности (xi) 0.79. При этом минимальное количество кластеров должно быть либо равно 2м, либо None. При увеличении количества кластеров результат ухудшается. Наилучший результат кластеризации все еще выдает Agglomerative clustering. DBSCAN, на которую похожа модель OPTICS, также выдает более точные результаты.
Также довольно сложно подбирать параметры.